## **LangChain -  FAISS**

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [ ]:
# 1) load PDF
pdf_path = 'Hayat-Qulub-Alama-Majlisi.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [ ]:
print(loader)

In [ ]:
# 2) Split into chunks 
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)

In [ ]:
# 3-A) Create Embeddings - Sentenace Transformer Embeddings
embeddings = HuggingFaceBgeEmbeddings(model_name='all-MiniLM-L12-v2', model_kwargs={'device':'cuda'})
embeddings

In [ ]:
# 4) Store Vector Embeddings
vectordb = FAISS.from_documents(chunks, embeddings)
vectordb.save_local("faiss_all_minilm")

In [ ]:
# 5) Retrieve 
retrieve = vectordb.as_retriever(search_type = 'similarity', search_kwargs={"k":10})
retrieve

In [ ]:
from dotenv import load_dotenv
# ---------- Load env var ----------
load_dotenv()

API_KEYS = os.getenv("GEMINI_API_KEYS", "").split(",")
API_KEYS = [k.strip() for k in API_KEYS if k.strip()]
current_key_index = 0
ACCESS_KEY = os.getenv("APP_ACCESS_KEY")

if not API_KEYS:
    print("❌ No Gemini API keys found. Check .env file.")
    raise Exception("Missing GEMINI_API_KEYS")

In [ ]:
# System Prompt
system_prompt = """
You are an Islamic history assistant. 
Always answer in a respectful and storytelling way. 
If the answer is not in the documents, say "I don’t know based on my knowledge."
Question: {question}
Context: {context}
Answer:
"""

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=system_prompt
)

In [ ]:
# 6) LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    api_key=os.getenv('GEMINI_KEY_KEY', '')
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retrieve,
    return_source_documents=False,
    chain_type_kwargs = {"prompt":prompt}
)

In [ ]:
# 7) Ask
query = "Why Adam was named Adam? Give reference to it as well"
res = qa.run(query)
print(res)

In [ ]:
q1 = "Why Adam was named Adam? Explain in details"
res = qa.run(q1)
print(res)

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

## **Pinecone Storage**

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeBaseClient
import os
from dotenv import load_dotenv

d:\Sadiq\rag-for-hq-books\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Sadiq\rag-for-hq-books\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [50]:
load_dotenv()
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ_V2")
index = pc.Index(INDEX_NAME)


In [49]:
index.delete(delete_all=True)

{}

### **Vectorinze Single PDF**

In [ ]:
# ---------- begin of pipelines ----------
# load pdf
pdf_path = 'Hayat-al-Qulub-Vol-1.pdf'
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

In [52]:
from langchain.schema import Document
import json
with open("hadiths_updated.json", "r", encoding="utf-8") as f:
    data = json.load(f)   # expecting a list of {"quote_no":1,"quote":"..."} or a wrapper object

# if JSON is a wrapper like {"quotes": [...]}
if isinstance(data, dict) and "hadith_text" in data:
    items = data["quotes"]
else:
    items = data

# docs = []

# for item in items:
#     text = item.get("hadith_text") or item.get("text") or item.get("body") or ""
#     meta = {"hadith_no": item.get("hadith_no"), **{k:v for k,v in item.items() if k not in ("hadith_text","text","body")}}
#     docs.append(Document(page_content=text, metadata=meta))


In [53]:
docs = []
for item in items:
    hadith_no = item.get("hadith_no")
    text = item.get("hadith_text","").strip()
    # append ID into the content
    content = f"{text}\n\n[HADITH_NO: {hadith_no}]"
    meta = {k:v for k,v in item.items() if k not in ("hadith_text",)}
    docs.append(Document(page_content=content, metadata=meta))

In [54]:
docs[0]

Document(metadata={'hadith_no': 0, 'section_type': 'preface', 'page_start': 1, 'page_end': 5, 'page_range': '1-5', 'char_count': 2500, 'word_count': 450, 'extracted_at': '2025-10-26T21:24:22.839229'}, page_content='Imam Ja’far Al Sadiq (a.s) said: “If anyone from our Shia and devotees does not have the book of Sulaym ibn Qays al Hilali, then he does not have any of our things, and he does not know any of our matters. This is the first book of Shia and is one of the secrets of Ale-Muhammad (a.s)” Introduction BISMILLAHIR RAHMANIR RAHEEM Wa Sallallahu ‘Ala Muhammad Wa Alahit Tayyabin al Muntakhabin And May Allah shower his blessings on the Holy Prophet (SAW) and his holy progeny who are pure and chosen ones. It is believed that Fazile Muhaddith Shaykh Shehr Ashoob Assarawi al Mazindarani, who was one of the greatest Ulamah in 565 A.H. has narrated that he read the hand written book of Sulaym ibn Qays Al Hilali to Raise Afif Abul Baqa Hibatullah ibn Nama ibn Ali ibn Hamdun (RA), in his ho

In [55]:
docs[1]

Document(metadata={'hadith_no': 1, 'page_start': 6, 'page_end': 9, 'page_range': '6-9', 'char_count': 6365, 'word_count': 1182, 'extracted_at': '2025-10-26T21:24:22.839229', 'section_type': 'hadith'}, page_content='Sulaym says: “I heard Salman al-Farsi saying: “I was sitting with the Holy \nProphet (SAW) while he was in that period of illness in which he passed \naway. Lady Fatimah (AS) entered and when she saw the weak state of the \nHoly Prophet’s (SAW) health, she got into the state of crying until tears \nstarted rolling down her cheeks. The Holy Prophet (SAW) asked: “Why are \nyou crying?” She replied “O Prophet of Allah after your death I am scared of \ndestruction of myself and my children.” The Holy Prophet (SAW) with his \neyes full of tears, said: “Oh Fatimah don’t you know? We are people of that \nhousehold which Allah has chosen for the hereafter instead of this world and \nHe ordained it necessary for all beings to be destroyed. And Allah Tabarak \nwa T’ala looked at the e

In [56]:
# split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)
chunks

[Document(metadata={'hadith_no': 0, 'section_type': 'preface', 'page_start': 1, 'page_end': 5, 'page_range': '1-5', 'char_count': 2500, 'word_count': 450, 'extracted_at': '2025-10-26T21:24:22.839229'}, page_content='Imam Ja’far Al Sadiq (a.s) said: “If anyone from our Shia and devotees does not have the book of Sulaym ibn Qays al Hilali, then he does not have any of our things, and he does not know any of our matters. This is the first book of Shia and is one of the secrets of Ale-Muhammad (a.s)” Introduction BISMILLAHIR RAHMANIR RAHEEM Wa Sallallahu ‘Ala Muhammad Wa Alahit Tayyabin al Muntakhabin And May Allah shower his blessings on the Holy Prophet (SAW) and his holy progeny who are pure and chosen ones. It is believed that Fazile Muhaddith Shaykh Shehr Ashoob Assarawi al Mazindarani, who was one of the greatest Ulamah in 565 A.H. has narrated that he read the hand written book of Sulaym ibn Qays Al Hilali to Raise Afif Abul Baqa Hibatullah ibn Nama ibn Ali ibn Hamdun (RA), in his h

In [57]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)

In [58]:
# Store in vector database - Pinecone
vector_db = PineconeVectorStore.from_documents(documents=chunks, embedding=embeddings, index_name=INDEX_NAME)

In [ ]:
# Store in vector database - FAISS
vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("Hayat-Qulub-Alama-Majlisi-faiss-index")

### **Vectorize multiple PDFs**

In [16]:
#  Store all 3 PDFs in vector database - Pinecone
pdfs = [
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-1.pdf',
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-2.pdf',
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-3.pdf'
]

all_chunks = []
for i, pdf in enumerate(pdfs):
    loader = PyMuPDFLoader(pdf)
    docs = loader.load()
    print(f"Loaded {i} documents")
    
    # split text into chunks
    print(f"Splitting {i} documents into chunks...")
    splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    chunks = splitter.split_documents(docs)
    print(f"splitted {i} documents Successfully! ")
    print(f"Loaded {len(chunks)} chunks from {i} document {pdf}")

    all_chunks.extend(chunks)
    print(f"Chunk list filled")
    print(f"Total chunks so far: {len(all_chunks)}")
    


Loaded 0 documents
Splitting 0 documents into chunks...
splitted 0 documents Successfully! 
Loaded 2547 chunks from 0 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-1.pdf
Chunk list filled
Total chunks so far: 2547
Loaded 1 documents
Splitting 1 documents into chunks...
splitted 1 documents Successfully! 
Loaded 3806 chunks from 1 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-2.pdf
Chunk list filled
Total chunks so far: 6353
Loaded 2 documents
Splitting 2 documents into chunks...
splitted 2 documents Successfully! 
Loaded 1205 chunks from 2 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-3.pdf
Chunk list filled
Total chunks so far: 7558


In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)


In [ ]:
# Store in vector database - Pinecone
vector_db = PineconeVectorStore.from_documents(documents=all_chunks, embedding=embeddings, index_name=INDEX_NAME)
pinecone_store = vector_db

In [19]:
pinecone_store = vector_db

### **Vectorize multiple PDFs - functions**

In [ ]:
def process_pdf_into_chunks(pdfs):
    pdfs = list(pdfs)
    all_chunks = []
    for i, pdf in enumerate(pdfs):
        loader = PyMuPDFLoader(pdf)
        docs = loader.load()
        print(f"Loaded {i} documents")
        
        # split text into chunks
        print(f"Splitting {i} documents into chunks...")
        splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        chunks = splitter.split_documents(docs)
        print(f"splitted {i} documents Successfully! ")
        print(f"Loaded {len(chunks)} chunks from {i} document {pdf}")

        all_chunks.extend(chunks)
        print(f"Chunk list filled")
        print(f"Total chunks so far: {len(all_chunks)}")
    
    return all_chunks

def store_chunks_in_pinecone(all_chunks, embedding_model, INDEX_NAME):
    # create embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model, model_kwargs={"device": "cuda"}
    )
    print(f"Created embeddings using model: {embedding_model}")
    
    # Store in vector database - Pinecone
    pinecone_store = PineconeVectorStore.from_documents(documents=all_chunks, embedding=embeddings, index_name=INDEX_NAME)
    print(f"Stored {len(all_chunks)} chunks in Pinecone index: {INDEX_NAME}")
    return pinecone_store


### **Retrieve QA**

In [9]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone as PineconeBaseClient
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

load_dotenv()

# Setup Pinecone
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
index = pc.Index(INDEX_NAME)

# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cpu"}
)

pinecone_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME, 
    embedding=embeddings)

# Create retriever from Pinecone
retrieve = pinecone_store.as_retriever(
    search_type='similarity', 
    search_kwargs={"k": 10}
)

# System Prompt
system_prompt = """
You are an Islamic history assistant. 
Always answer in a respectful and storytelling way. 
You have to understand the context deeply and answer accordingly.
If the answer is not in the documents, say "I don't know based on my knowledge."
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(system_prompt)

# LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    temperature=0,
    api_key=os.getenv('GEMINI_API_KEY')
)

# llm = ChatOpenAI(
#     model_name='gpt-4o', 
#     temperature=0,
#     api_key=os.getenv('OPEN_AI_KEY')
# )

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

input = {"context": retrieve | format_docs, 
      "question": RunnablePassthrough()
      }


qa_chain = ( input| prompt | llm | StrOutputParser())


In [ ]:
# Ask a question
query = "Why Adam was named Adam? Give reference to it as well"
res = qa_chain.invoke(query)

print(res)

In the tapestry of Islamic history, the naming of Adam is a tale woven with profound meaning.

It is narrated through authentic chains that Imam Muhammad al-Baqir and Ja‘far as-Sadiq (peace be upon them) said, "Adam was named ‘Adam’ because he was ‘Adeemul Arz’, that is he was created from the face of the earth (from dust)."

So, Adam's name is intrinsically linked to his origin, a reminder of the humble earth from which he was formed.


In [13]:
query1 = """
Kisne likh hai Hayatal Qulub? kiske bare me hai ye kitab? 
kitni jildhe hai iske aur kya har ek jildh ko short me smjha sakte ho?
"""
res2 = qa_chain.invoke(query1)
print(res2)

Aapka sawaal Hayat-ul-Qulub ke baare mein hai. Chaliye, main aapko iske baare mein tafseel se batata hoon:

**Hayat-ul-Qulub kisne likhi hai?**

Hayat-ul-Qulub Allama Muhammad Baqir Majlisi (Rehmatullah Alaih) ne likhi hai. Allama Majlisi ek mashhoor Islamic scholar aur Muhaddith the.

**Ye kitab kiske baare mein hai?**

Ye kitab Ambiya (Prophets), Aimma (Imams), aur deegar buzurg hastiyon ke ahwaal (life events) aur fazail (virtues) par mabni hai. Isme unke zindagi ke waqiyat, unke akhlaq, aur unke maqamat ko tafseel se bayan kiya gaya hai.

**Is kitab ki kitni jildhein (volumes) hain?**

Hayat-ul-Qulub ki teen jildhein hain. Har jildh mein mukhtalif anbiya aur aimma ke baare mein tafseeli malumaat hain.

**Har jildh ka mukhtasar khulasa (brief summary):**

*   **Jildh 1:** Is jildh mein Ambiya-e-Kiram (Prophets) ke ahwaal bayan kiye gaye hain, jaise Hazrat Adam (A.S.), Hazrat Nuh (A.S.), Hazrat Ibrahim (A.S.), Hazrat Musa (A.S.), aur Hazrat Isa (A.S.). Har Nabi ki zindagi ke aham waq

In [ ]:
# quick check for any embedding wrapper you plan to use
vec = embeddings.embed_query("test")
print("type:", type(vec))
print("len:", len(vec))   # MUST be 1024 to match your Pinecone index

## **Sulaym ibne Qays RAG**

### **RAG Pipeline**

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeBaseClient
from dotenv import load_dotenv
import os

c:\Users\sadiq\OneDrive\Documents\projects\rag-for-Hayatal-Qulub\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [6]:
load_dotenv()

pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ")
index = pc.Index(INDEX_NAME)

In [33]:
def process_pdf_into_chunks(pdfs: list):
    print(f'Count of Documents - {len(pdfs)}')

    all_chunks = []
    for i, pdf in enumerate(pdfs):
        i=+1
        loader = PyMuPDFLoader(pdf)
        docs = loader.load()
        print(f"Loaded {i} document successfully")

        # split into chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        chunks = splitter.split_documents(docs)
        print(f"Splitted {i} documment successfully!")

        all_chunks.extend(chunks)
        print(f"Total chunks so far: {len(all_chunks)}")

    return all_chunks
    
def store_chunks_in_pinecone(all_chunks, embedding_model, INDEX_NAME):
     #create embeddings

    embeddings = HuggingFaceEmbeddings(
        model_name = 'BAAI/bge-large-en-v1.5', model_kwargs={"device": "cuda"} 
        )
    print(f"Created embeddings using model: {embedding_model}")

    # Store in vector database - Pinecone
    pinecone_store = PineconeVectorStore.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        index_name=INDEX_NAME
    )
    print(f"Stored {len(all_chunks)} chunks in Pinecone index: {INDEX_NAME}")
    return pinecone_store
    

In [34]:
chunks = process_pdf_into_chunks(pdfs=['kitab_sulaim_ibn_qays_al-hilaali.pdf'])

Count of Documents - 1
Loaded 1 document successfully
Splitted 1 documment successfully!
Total chunks so far: 753


In [35]:
vector_store = store_chunks_in_pinecone(chunks, 'BAAI/bge-large-en-v1.5', INDEX_NAME)

Created embeddings using model: BAAI/bge-large-en-v1.5
Stored 753 chunks in Pinecone index: sulaym-ibne-qays


### **Retrieve QA**

In [4]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone as PineconeBaseClient
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Markdown

from system_prompt import system_prompt_text
load_dotenv()

# Setup Pinecone
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ_V2")
index = pc.Index(INDEX_NAME)

# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cpu"}
)

pinecone_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME, 
    embedding=embeddings)

# Create retriever from Pinecone
retrieve = pinecone_store.as_retriever(
    search_type='similarity', 
    search_kwargs={"k": 10}
)

# System Prompt
system_prompt = system_prompt_text

prompt = ChatPromptTemplate.from_template(system_prompt)

# LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    temperature=0,
    api_key=os.getenv('GEMINI_API_KEY')
)

# llm = ChatOpenAI(
#     model_name='gpt-4o', 
#     temperature=0,
#     api_key=os.getenv('OPEN_AI_KEY')
# )

def format_docs(docs):
    blocks = []
    for d in docs:
        hadith_no = d.metadata.get("hadith_no", "UNKNOWN")
        page_start = d.metadata.get("page_start")
        # ensure hadith_no is integer-like for neatness
        try:
            hadith_tag = f"[HADITH_NO:{int(hadith_no)}]"
        except:
            hadith_tag = f"[HADITH_NO:{hadith_no}]"
        # clean page markers inside content (optional)
        content = d.page_content.replace("(Page", "").replace("Page", "")
        content = content.strip()
        blocks.append(f"{hadith_tag} | PAGE:{page_start}\n{content}")
    return "\n\n".join(blocks)


input = {"context": retrieve | format_docs, 
      "question": RunnablePassthrough()
      }


qa_chain = ( input| prompt | llm | StrOutputParser())

def ask(question: str) -> str:
    """Ask a question and get an answer from the RAG system."""
    result = qa_chain.invoke(question)
    return result

In [82]:
# # debug inspect retrieved docs
# retrieved = retrieve.get_relevant_documents(query)
# for i, d in enumerate(retrieved[:8]):
#     print(f"--- RESULT {i} ---")
#     print("metadata keys:", list(d.metadata.keys()))
#     print("hadith_no:", d.metadata.get("hadith_no"))
#     print("page_no:", d.metadata.get("page_start") or d.metadata.get("page_no") or d.metadata.get("page"))
#     print("first 200 chars:", d.page_content[:200].replace("\n"," "))
#     print()


In [83]:
# res = index.fetch(ids=["1"])   # try an id you expect to exist
# print(res)   # look under 'vectors' -> id -> metadata

In [5]:
from IPython.display import display, Markdown
# Ask a question
query = "Few people attacked the house of Lady Fatimah (SA) and burned the door. what was the entire incident? Explain in details with hadith no."
#res = qa_chain.invoke(query)
res = ask(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that...

After the Holy Prophet (SAWA) passed away, Umar gathered people to force Imam Ali (AS) to pledge allegiance to Abu Bakr. They went to Imam Ali's (AS) house, where Lady Fatimah (SA) and their two sons were. Umar ordered his companions to collect wood and pile it around the house [HADITH_NO:4].

Umar then shouted loudly, so that Imam Ali (AS) and Lady Fatimah (SA) could hear: "By God, O Ali, come out and pay allegiance to the caliph of the Holy Prophet of Allah, otherwise we will burn your house" [HADITH_NO:4].

Lady Fatimah (SA) asked: "O Umar, what do you have to do with us?" [HADITH_NO:4].

Umar replied: "Open the door, otherwise we will burn your house" [HADITH_NO:4], [HADITH_NO:48].

Lady Fatimah (SA) said: "O Umar, are you not afraid of Allah and are you entering our house?" [HADITH_NO:4].

Umar refused to return. He asked for fire to be brought and he set the door on fire, then he pushed it and entered. Lady Fatimah (SA) came in front and screamed loudly: "O Father, O Prophet of Allah" [HADITH_NO:4], [HADITH_NO:48].

According to the traditions, Janabe Sulaym ibne Qays narrated from Salman that...

Salman said: "Yes, by God, when she did not even have a chaddar over her. So she screamed: “O Father, O Prophet of Allah, Abu Bakr and Umar behaved so badly after you, while your eyes have not even closed in the grave” and she was saying this loudly" [HADITH_NO:4].

According to the traditions, Janabe Sulaym ibne Qays narrated that...

Umar raised his sword with the shield and hit her on the side. She screamed [HADITH_NO:4].

References: [HADITH_NO:4], [HADITH_NO:48]

In [2]:
query = "Explain the Hadith no 4 exactly in detail"
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that after the demise of the Prophet Muhammad (PBUH), he was present when a man from the Ansar spoke to a group of people, including himself, about the events that transpired. The man said:

"When the Messenger of Allah (PBUH) passed away, Ali (AS) was occupied with his burial. A group of people gathered in the Saqifa of Bani Sa'ida to exclude him from the affair (of leadership). When Ali (AS) finished with him (the Prophet (PBUH)), he sat at home and gathered the Quran, for he had vowed to Allah that he would not put on his outer garment except for prayer until he had gathered it. Then, Abu Bakr sent word to him, 'Come and pledge allegiance.' But Ali (AS) sent back to him, 'I am occupied.' Abu Bakr sent word to him again, 'Come, or I will come to you and fight you.' Then Umar (LA) came with a group and set fire to Ali's (AS) door. Fatima (SA) came out and said, 'O Umar, what is this?' He said, 'Why don't people enter for allegiance?' Then they took Ali (AS) forcefully and brought him to Abu Bakr." [HADITH_NO:4]

The man from the Ansar continued, "Ali (AS) said to Abu Bakr, 'O Abu Bakr, you are more entitled to this affair than I am. I will obey you and give you allegiance, but tell me, O Muhajirun, is there anyone among you who is closer to the Messenger of Allah (PBUH) than I am? Or who has more knowledge of the Quran and the Sunnah than I do? Or who is more deserving of this affair than I am?' They said, 'No, by Allah.' Then Ali (AS) said, 'I ask you by Allah, O Muhajirun, did you hear the Messenger of Allah (PBUH) say on the Day of Ghadir Khumm, "Whoever I am his master (Mawla), then Ali is his master (Mawla). O Allah, befriend whoever befriends him and be an enemy to whoever is his enemy; and support whoever supports him and abandon whoever abandons him?"' They said, 'Yes, by Allah, we heard it.' Ali (AS) said, 'Then why did you dispute with me over this affair?' They said, 'We feared that you would not be able to handle it.'" [HADITH_NO:4]

The man from the Ansar further narrated, "Then Ali (AS) said, 'By Allah, I am more capable of handling it than you are. But I will obey you as long as I live.' Then Ali (AS) gave allegiance to Abu Bakr." [HADITH_NO:4]

References: [HADITH_NO:4]

In [5]:
query = "Who are those 3 types of people who understand and accept the Ahle bait (as)?"
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated from Ali ibn Abi Talib (AS) that the Holy Prophet (PBUH) said: ["You people fear three types of people relating to your religion. One is that person who recited the Quran until he saw its splendour on him like a cover of faith (Iman) - he changed it to a great extent, so he raised his sword on his Muslim brother and started calling him idol worshipper."] So I asked: "O Messenger of Allah, who is more deserving to be called idol worshiper out of the two?" He replied: ["The one who accused the other of doing shirk (i.e. associating a partner to God) [HADITH_NO:54].

According to the traditions, Janabe Sulaym ibne Qays narrated that there are three types of people: "- a believer who knows our rights, believes in what we say, and in every matter keeps us ahead. This is the person who is saved and is honest and is Allah’s friend. Whoever established enmity with us and kept away from us, curses us and considers our blood permitted, and denies our rights and has made keeping away from us religion - this person is an infidel, an idolater and cursed. The one who chooses that in which there is no dispute and whatever he finds difficulty in, he returns its knowledge to Allah, even if it is our Wilayat, and does not bear enmity towards us, we are hopeful for him and his decision is in Allah’s hands [HADITH_NO:42].

According to the traditions, Janabe Sulaym ibne Qays narrated that Imam Ali (AS) said: “one type is eating through us and one is that who has been guided through us and obeyed our Amr. These are very few in this type. These are those who are Shias, noble, people of patience, are learned, jurist, pious and generous. For them there is happiness and also a tree in heaven, and wherever they go, the place will be very good [HADITH_NO:79].”

According to the traditions, Janabe Sulaym ibne Qays narrated that Imam Ali (AS) said: “Those who know our rights and have faith in us, they are all Mumin, Muslims and are friends of Allah [HADITH_NO:70].”

References: [HADITH_NO:54], [HADITH_NO:42], [HADITH_NO:79], [HADITH_NO:70]

In [5]:
# Ask a question
query = """
Give me 5 merits of Imam Ali (AS) in Islam with references.
"""
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that:

1.  Imam Ali (AS) was the first to accept Islam. All the companions of the Holy Prophet (S.A.W) had no doubt, nor disagree, and there was no dispute that Ali (AS) was the first to accept Islam [HADITH_NO:58].
2.  Imam Ali (AS) was more knowledgeable than all the other companions. All the companions of the Holy Prophet (S.A.W) had no doubt, nor disagree, and there was no dispute that Ali (AS) was the first to accept Islam and he was more knowledgeable than all of them [HADITH_NO:58].
3.  Imam Ali (AS) tolerated more difficulties fighting in the way of Allah and defending from enemies and saving the Holy Prophet (S.A.W) at the cost of his own life [HADITH_NO:58].
4.  The Holy Prophet (S.A.W) said to Imam Ali (AS): "O Ali, you are brother, you are friend, you are successor, you are vizier, you are caliph in family and wealth and at all time when I am not there. And you have the same status to me that I have with my God. You are caliph in my Ummah. Your friend is my friend, and your enemy is my enemy. You are Amirul Mumineen and Sayyidul Muslimeen after me" [HADITH_NO:40].
5.  The Holy Prophet (S.A.W) said: "Do not upset me concerning my brother Ali because he is Amirul Mumineen, Sayyidul Mursleen and is the bearer of Livai Hamd (Standard bearer) and on the Day of Judgement he is Qaedul Gurril Muhajjileen (the leader of those whose forehead will be shining). Allah will nominate him on Sirat (The Path) and he will divide the Hell. He will enter his friends in Heaven and enemies in Hell" [HADITH_NO:24].

References: [HADITH_NO:58], [HADITH_NO:40], [HADITH_NO:24]

In [44]:
from IPython.display import display, Markdown
display(Markdown(res))

From the narrations in Kitab Sulaym ibn Qays, the merits of Imam Ali (AS) are numerous and profound. Here are five, illuminated by the traditions within this esteemed collection:

1.  **The First to Accept Islam:** Imam Ali (AS) was the first among the Ummah to embrace Islam. As Imam Ali (AS) himself states, referencing the verse "{'And the foremost are the foremost, These are drawn nigh (to Allah)' } (Surah Waqia: 10- 11)," no one preceded him in turning towards Allah and His Prophet (PBUH). This precedence is not merely a matter of historical record, but a testament to his unwavering faith and devotion from the earliest days of Islam.

2.  **Unmatched Knowledge and Jurisprudence:** After the passing of the Holy Prophet (PBUH), all were dependent on Imam Ali (AS) for knowledge and jurisprudence, while he remained independent of all others. The Holy Prophet (PBUH) himself imparted knowledge to Imam Ali (AS) privately, day and night, ensuring that he was deeply versed in the Book of Allah and the traditions of the Prophet (PBUH).

3.  **Bravest in Wars and Defender of the Prophet (PBUH):** Imam Ali (AS) was always at the forefront, trusted by the Holy Prophet (PBUH) due to his virtues. No difficulty befell the Holy Prophet (PBUH), no problems arose for the Holy Prophet (PBUH) defending against bravery, conquering any fort, but Ali was always sent in forefront. He consistently risked his life to protect the Holy Prophet (PBUH) and defend the nascent Muslim community.

4.  **The Brother, Vizier, and Successor of the Prophet (PBUH):** Allah sent a revelation to the Holy Prophet (PBUH) to make Imam Ali (AS) his brother, friend, vizier, successor, and caliph. The Holy Prophet (PBUH) declared that Imam Ali (AS) is, after him, the guardian of all believers. This appointment signifies Imam Ali's (AS) divinely ordained role in guiding the Ummah after the Prophet (PBUH).

5.  **The Organizer, Witness, and Guide of the Ummah:** The Holy Prophet (PBUH) said that Ali (AS) is the organizer of this Ummah and is a witness over it and is responsible for its account, and that is the greatest Sign, and is very illuminated road of truth and is Allah's Path. After the Holy Prophet (PBUH), guidance can be got from him after deviation, and through him can the light be achieved from blindness. This highlights Imam Ali's (AS) role as the guiding light for the Ummah, leading them from darkness to enlightenment.

In [72]:
# Ask a question
query = "Tell me about the 73rd sect which will go to paradise with references."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that Imam Ali ibn Abi Talib (AS) said: “The Ummah will be separated in 73 sects, of which 72 will go to Hell, and 1 will go to Heaven." (Page 7)

According to the traditions, Janabe Sulaym ibne Qays narrated that Imam Ali (AS) said: "From the 73, 13 will show their love towards us Ahlulbayt, from which 1 will go to Heaven and 12 will go to Hell. The sect that will be saved will be the one which is guided, which is hopeful, which has Iman, obeys command (bows his head), is agreeable and intelligent. It is he that trusts me and bows his head to my command, obeys my command, chooses to keep away from my enemy, loves me, keeps enmity with my enemy, has recognised my rights, and Imamat and has realised my obedience compulsory through the Book of Allah and the Tradtion of the Holy Prophet, and has not become an apostate, and has not doubted, because Allah has enlightened his heart with the knowledge of our rights and has recognised our merits and all this has Quran with us. We will not separate from it and it will not separate from us until we reach the Heavenly Fountain of the Holy Prophet (S.A.W) as he has said: And this is the one sect, out of the 73, that will be saved from Hell and will be saved from discord, misguidance and doubts, and this, surely, is from the dwellers of Heaven and these are 70000 who will enter Heaven without being questioned. As 72 sects have chosen false Deen, and are helpers of the deen of Shaytan and have taken that deen from Shaytan and its friends. These are enemies of Allah, His Prophet, and the faithful, and will go to Hell without any reckoning (hisab). These people will be far from Allah and His Prophet – they have forgotten Allah and His Prophet, have associated partners to Allah, are idolaters and have denied Him, and except God, have worshipped others in a manner that they do not even know. They think that idolaters will go to Hell without questions. Reckoning will only be for those types of people who are in between the faithful and idolaters, and muallafatul qulub (those whose hearts can be attracted) and those who are mukhtnfah (who have a little knowledge) and those who, along with good deeds, have done bad also and are with weak faith who are confused about kufr and shirk and do not really know whether to ebar enmity and will not even choose that Path which will make them Mumin or Aarif. These people will be from Ashab-‘Aaraf. These will be those at Allah’s discretion. If Allah sends him to Hell, then that will be due to sins, and if He forgives then it will be His Mercy.” (Pages 7-8)

According to the traditions, Janabe Sulaym ibne Qays narrated that Imam Ali (AS) said: "But those who believe in Tawheed and have brought Iman on the Holy Prophet (S.A.W) but do not know our Wilayat and do not know the misguidance of our enemies, have not made halal or haram, and have chosen those things in which there is no disagreement, and which Allah commanded. In matters where there is disagreement in the Ummah, they stay away, not sure of whether it has been commanded or not – so they have not chosen to be enemy or have not made lawful or unlawful, have no knowledge and whatever difficulty they come across, they leave it to Allah. So they will be saved. This sect is in-between the faithful and idolater and majority of people are in there - myself, my brother Ali, the best of all and my best friend, and Fatimah who will be the leader of the entire women of Paradise, Hasan and Husayn and 9 Imams from the children of Husayn. We will be 14 people in one place. Allah has kept all evil away from us, and has purified us in such a manner that we deserve to be purified. These will be the guides and guided ones. I am the conveyer from Allah and each one of these are conveyors from myself and Allah ‘Azz wa Jall, and these are all the proofs of Allah Tabarak wa T’Ala over His creation and are His witnesses on His earth and are treasures of His knowledge and mines of wisdom. Whoever obeys them will have obeyed Allah and whoever disobeys will have disobeyed Allah. The earth cannot remain even for a split second until they do not remain, and without them there can be no order on the earth. They are the informers of the Deen to Ummah and are informers of lawful and unlawful and explain and the Book will be with them. They will not separate from the Book and the Book will not separate from them until they all reach the Fountain of the Holy Prophet (S.A.W). We are that ahlulbayt for whom our father Ibrahim prayed and said: {'therefore make the hearts of some people yearn towards them'} (Surah Ibrahim: 37) So particular reference has been to us, and we are those people whom Allah has meant by {'O you who believe, bow down and prostrate yourselves and serve your Lord, and do good that you may succeed. And strive hard in (the way of) Allah, (such) a striving as is due to Him; He has chosen you and has not laid upon you any hardship in religion; the faith of your father Ibrahim; He named you Muslims before and in this, that the Apostle may be a bearer of witness to you, and you may be bearers of witness to the people, therefore keep up prayer and pay the poor-rate and hold fast by Allah; He is your Guardian, so what an excellent Guardian and what an excellent Helper!'} (Surah Hajj: 77-78)." (Pages 7, 42, 54)

According to the traditions, Janabe Sulaym ibne Qays narrated that the Holy Prophet (SAW) said: “The Jews have 71 sects, from which 70 will go to Hell and one will go to Heaven and it is the sect that followed the wasi and Musa (AS). The Christians have 72 sects, 71 will go to Hell and one will go to Heaven and it is that sect who followed Isa’s wasi, and my Ummah will have 73 sects, 72 will go to Hell and one will go to Heaven and that sect is the one that follows my wasi.” (Page 65)

According to the traditions, Janabe Sulaym ibne Qays narrated that the Holy Prophet (SAW) said: “13 sects from 73 will all give connections to my love and devotion from them, 1 will be in Heaven and 12 in Hell.” (Page 65)

According to the traditions, Janabe Sulaym ibne Qays narrated that Imam Ali (AS) said to the leader of Jews: “How many sects have happened amongst you?” He replied: “This many and this many.” Ali (AS) said: “You have told a lie.” Then he turned to people and said: “By God, if a cushion is kept for me I would make a judgment based on their Torah among the people of Torah and on their Injil amongst the people of Injil and on Quran amonst people of Quran. Jews have become 71 sects - 70 sects from them will be in Hell, 1 sect will go to Heaven and this is the one that obeyed Yush’a ibn Noon, the trustee of Musa and Christians are divided into 72 sects. 71 from them will go to Hell and 1 in Heaven and this is the one who obeyed Sham’un, the trustee of Isa (AS). This Ummah will have 73 sects - 72 from it will go to Hell and 1 will go to Heaven, and this the sect that obeyed the successor of Muhammad (SAW). (Pages 32-33)

In [46]:
# Ask a question
query = "Tell me bout diff between imaan n islam as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

Sulaym ibn Qays narrates a profound discussion on the essence of Iman and Islam, a conversation he had directly with Imam Ali (AS). He asked Ameerul Momineen (AS), "What is Iman and Islam?"

Imam Ali (AS) illuminated the distinction, stating, "Iman is to understand with knowledge, and accept, and Islam is to accept, bow your head down and obey."

Sulaym then sought further clarification, asking, "Does Iman mean to accept after knowledge (Manfah)?"

Imam Ali (AS) affirmed this, explaining, "He, whom Allah has given knowledge about HIM, His Prophet (PBUH) and His Imam (AS), then he has accepted obedience to Allah – he is a Mumin."

Sulaym then summarized his understanding, "That means from Allah is knowledge (Manfah) and from human being is acceptance."

Imam Ali (AS) confirmed this understanding, elaborating, "Manfah from Allah means that He calls, and gives authority and obliges and gives bounties, and acceptance of Allah means that the human-being accepts that He is free to oblige whomever He pleases and He develops Manfah in whoever’s heart He wishes."

This narration highlights the critical role of both divine guidance and human acceptance in achieving true faith. Knowledge bestowed by Allah (SWT) through His Prophet (PBUH) and the Imams (AS) is the foundation, but it is the individual's conscious acceptance and obedience that completes the essence of Iman. Islam, in this context, is the outward manifestation of that inner conviction, expressed through submission and adherence to Allah's (SWT) commands.

In [ ]:
# Ask a question
query = "Tell me bout the accursed clan as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the narrations in Kitab Sulaym ibn Qays, the animosity towards the Ahlulbayt (AS) is deeply rooted in the hearts of some, stemming from events like the battles of Badr and Uhud. Imam Ali (AS) spoke of those who harbor malice and seek to overpower the rightful successors of the Prophet (PBUH).

Sulaym ibn Qays narrates that Imam Ali (AS) described the state of affairs after the passing of the Prophet (PBUH), saying that the Quraysh had overpowered him and that Umar and his companions had prepared Abu Bakr and paid allegiance to him while he (AS) was occupied with the sacred duties of preparing the Prophet (PBUH) for burial. He (AS) further stated that he reminded the people of his rights and sought their help, but they neither answered nor aided him.

The book also mentions Amr ibn al-Aas, who, after the passing of Ibrahim, the son of the Prophet (PBUH), spoke ill of the Prophet (PBUH) and his lineage. This prompted the revelation of Surah Al-Kauther, which assures the Prophet (PBUH) that his enemy will be without posterity.

Imam Hasan (AS) was forewarned about the treachery he would face from the Ummah. He was advised to fight if he found helpers, but if not, to restrain himself and preserve his blood, as martyrdom awaited him. The Imam (AS) was also informed that his killer would be an illegitimate child, cursed by Allah.

These narrations paint a picture of the trials and tribulations faced by the Ahlulbayt (AS) and their followers, highlighting the importance of remaining steadfast in the face of adversity.

In [49]:
# Ask a question
query = "Tell me about the accursed tree, shajra e maloon as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))      

The "cursed tree" mentioned in the verse, as Imam Ali (AS) explains, is a trial for mankind. Sulaym ibn Qays narrates that Imam Ali (AS) spoke of the Quraysh and their oppression towards them, the Ahlulbayt (AS). He mentioned the verse, "And We did not make the vision which We showed you but a trial for men and the cursed tree in the Qur'an."

This "cursed tree," according to the Imam (AS), refers to those who would usurp the rights of the Ahlulbayt (AS) after the Prophet's (PBUH) passing. They are the ones who would lead people astray and cause great harm to the Ummah. The Holy Prophet (PBUH) informed Imam Ali (AS) that he saw twelve misguiding imams from Quraysh climbed up and down his pulpit, whose faces were like monkeys who are taking his Ummah from Siratul Mustaqeem (the Right Path) back to whom they were.

In [50]:
# Ask a question
query = "Tell me bout imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

Sulaym ibn Qays narrates a remarkable event that highlights the unique status of Imam Ali (AS). One morning, Amirul Momineen (AS) went to Baqi' with Abu Bakr, Umar, and a community of Muhajireen and Ansar. He stood on a high place, and as the sun rose, he addressed it, saying, "Salutation, O creature of Allah’s new creation who obeys Him."

According to this tradition, everyone present heard a voice from the sky responding, "And salutation be on you as well, O that person, who is Awwal and O he, who is Aakheer, O Zahir, O batin, O he who knows everything."

The people were astonished and fainted upon hearing this. When they regained consciousness, they went to the Holy Prophet (PBUH) and questioned him about Imam Ali (AS), saying, "You say that Ali is a man like us, when the sun addressed him in a manner that Allah has addressed Himself."

The Holy Prophet (PBUH) then confirmed the truth of what they had heard. He explained that the sun's words were accurate:

*   "O Awwal": He is the first one who brought faith to me.
*   "O Aakheer": He is the last person of all who will separate from me. He will give me the ritual bath, shroud, and will enter me in my grave.
*   "O Zahir": All my knowledge became Zahir (clear) for him.
*   "O Batin": All my secrets are present with him.
*   "O he, who knows everything": He knows what is lawful and unlawful, and compulsory, tradition, and like it.

This narration underscores the profound connection between the Holy Prophet (PBUH) and Imam Ali (AS), and the immense knowledge and spiritual status bestowed upon him.

In [51]:
# Ask a question
query = "Tell me bout imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali also quote the entire incident."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

Sulaym ibn Qays narrates a remarkable event that highlights the unique station of Imam Ali (AS) in the eyes of Allah (SWT) and the Holy Prophet (PBUH).

One morning, after the Fajr prayer, Imam Ali (AS) emerged with Abu Bakr, Umar, and a group of Muhajireen and Ansar. They proceeded to Baqi and Imam Ali (AS) stood upon a high place. As the sun rose, Imam Ali (AS) addressed it, saying, "Salutation, O creature of Allah's new creation who obeys Him."

To the astonishment of all present, a voice from the sky responded, "And salutation be on you as well, O that person, who is Awwal and O he, who is Aakheer, O Zahir, O Batin, O he who knows everything."

Upon hearing this, Abu Bakr, Umar, and many of the Muhajireen and Ansar fainted. When they regained consciousness, they were bewildered and sought an explanation from the Holy Prophet (PBUH). They exclaimed, "You say that Ali is a man like us, when the sun addressed him in a manner that Allah has addressed Himself."

The Holy Prophet (PBUH) inquired, "What did you hear from it?"

They replied, "We heard it saying 'Salutations be on you, O he who is Awwal.'"

The Holy Prophet (PBUH) confirmed, "It said truth. He is the first one who brought faith to me."

They continued, "We heard it say: 'O Aakheer'."

He replied, "It said truth, he is the last person of all who will separate from me. He will give me the ritual bath, shroud and will enter me in my grave."

They added, "We heard it say: 'O Zahir'."

He replied: "It said truth. All my knowledge became Zahir (clear) for him."

They continued, "We heard it say 'O Batin'."

He replied: "It said truth. All my secrets are present with him."

Finally, they said, "We heard it say 'O he, who knows everything'."

He replied, "It said truth. He knows what is lawful and unlawful, and compulsory, tradition and like it."

This incident, as narrated by Sulaym ibn Qays, serves as a powerful testament to the divinely ordained status of Imam Ali (AS) and the profound knowledge and secrets entrusted to him by the Holy Prophet (PBUH). It underscores his unique position as the first to believe, the last to part, and the inheritor of the Prophet's (PBUH) manifest and hidden knowledge.

In [ ]:
# Ask a question
query = "Tell me bout imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali also quote the entire incident and give hadees no and page number from the book."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

Sulaym ibn Qays narrates a remarkable event that highlights the unique station of Imam Ali (AS) in the eyes of Allah (SWT) and His Messenger (PBUH).

One morning, after the Fajr prayer, Imam Ali (AS) emerged, accompanied by Abu Bakr, Umar, and a group of Muhajireen and Ansar. They proceeded to Baqi and Imam Ali (AS) stood upon a high place. As the sun rose, Imam Ali (AS) addressed it, saying, "Salutation, O creature of Allah's new creation who obeys Him."

To the astonishment of all present, a voice from the sky responded, "And salutation be on you as well, O that person, who is Awwal and O he, who is Aakheer, O Zahir, O Batin, O he who knows everything."

Upon hearing this, Abu Bakr, Umar, and many of the Muhajireen and Ansar fainted. When they regained consciousness, they were bewildered and sought an explanation from the Holy Prophet (PBUH). They exclaimed, "You say that Ali is a man like us, when the sun addressed him in a manner that Allah has addressed Himself."

The Holy Prophet (PBUH) inquired, "What did you hear from it?"

They replied, "We heard it saying 'Salutations be on you, O he who is Awwal.'"

The Holy Prophet (PBUH) confirmed, "It said truth. He is the first one who brought faith to me."

They continued, "We heard it say: 'O Aakheer'."

He replied, "It said truth, he is the last person of all who will separate from me. He will give me the ritual bath, shroud and will enter me in my grave."

They added, "We heard it say: 'O Zahir'."

He replied, "It said truth. All my knowledge became Zahir (clear) for him."

They then said, "We heard it say 'O Batin'."

He replied, "It said truth. All my secrets are present with him."

Finally, they said, "We heard it say 'O he, who knows everything'."

He replied, "It said truth. He knows what is lawful and unlawful, and compulsory, tradition and like it."

This incident, as narrated by Sulaym ibn Qays, serves as a powerful testament to the divinely ordained status of Imam Ali (AS) and the profound knowledge and secrets entrusted to him by the Holy Prophet (PBUH). It underscores his unique position as the first to believe, the last to bid farewell, and the inheritor of the Prophet's (PBUH) manifest and hidden knowledge.

## **LlamaIndex**

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader 
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
docs = SimpleDirectoryReader('pdf_docs').load_data()
docs


In [ ]:
# splitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter

In [ ]:
embeddings = HuggingFaceEmbedding(model_name="all-MiniLM-L12-v2")
embeddings

In [ ]:
storage_context = StorageContext.from_defaults(index_store=embeddings)
storage_context

In [ ]:
vector_store = VectorStoreIndex.from_documents(docs, storage_context=storage_context)
vector_store